In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install qrcode[pil]

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import tempfile
import tkinter as tk
from tkinter import filedialog, messagebox
from PyPDF2 import PdfFileReader
import qrcode
from PIL import Image, ImageTk
import zipfile

class PDFtoQRCodeApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PDF to QR Code Converter")
        self.create_widgets()
        
    def create_widgets(self):
        # File upload section
        self.upload_button = tk.Button(self.root, text="Upload PDF", command=self.upload_file)
        self.upload_button.pack(pady=10)

        # QR code customization options
        self.size_label = tk.Label(self.root, text="QR Code Size:")
        self.size_label.pack()
        self.size_entry = tk.Entry(self.root)
        self.size_entry.pack()

        self.error_correction_label = tk.Label(self.root, text="Error Correction Level (L, M, Q, H):")
        self.error_correction_label.pack()
        self.error_correction_entry = tk.Entry(self.root)
        self.error_correction_entry.pack()

        # QR code preview section
        self.preview_label = tk.Label(self.root, text="QR Code Preview:")
        self.preview_label.pack()
        self.preview_canvas = tk.Canvas(self.root, width=300, height=300)
        self.preview_canvas.pack(pady=10)

        # Download button
        self.download_button = tk.Button(self.root, text="Download QR Code(s)", command=self.download_qrcodes)
        self.download_button.pack(pady=10)

    def upload_file(self):
        self.filepath = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if self.filepath:
            self.generate_qrcodes()

    def generate_qrcodes(self):
        try:
            # Read PDF
            pdf_reader = PdfFileReader(self.filepath)
            self.qrcodes = []

            for page_num in range(pdf_reader.numPages):
                page = pdf_reader.getPage(page_num)
                text = page.extractText()

                # Generate QR code
                size = int(self.size_entry.get() or 300)
                error_correction = self.error_correction_entry.get().upper() or 'M'
                ec_levels = {
                    'L': qrcode.constants.ERROR_CORRECT_L,
                    'M': qrcode.constants.ERROR_CORRECT_M,
                    'Q': qrcode.constants.ERROR_CORRECT_Q,
                    'H': qrcode.constants.ERROR_CORRECT_H
                }
                ec_level = ec_levels.get(error_correction, qrcode.constants.ERROR_CORRECT_M)

                qr = qrcode.QRCode(
                    version=1,
                    error_correction=ec_level,
                    box_size=10,
                    border=4,
                )
                qr.add_data(text)
                qr.make(fit=True)

                img = qr.make_image(fill='black', back_color='white')
                self.qrcodes.append(img)

            self.show_preview()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def show_preview(self):
        if self.qrcodes:
            qr_img = self.qrcodes[0].resize((300, 300), Image.ANTIALIAS)
            self.qr_photo = ImageTk.PhotoImage(qr_img)
            self.preview_canvas.create_image(0, 0, anchor=tk.NW, image=self.qr_photo)

    def download_qrcodes(self):
        if not self.qrcodes:
            messagebox.showwarning("No QR Codes", "No QR codes generated to download.")
            return

        save_path = filedialog.askdirectory()
        if not save_path:
            return

        try:
            temp_dir = tempfile.mkdtemp()
            zip_path = os.path.join(temp_dir, "qrcodes.zip")
            with zipfile.ZipFile(zip_path, 'w') as zipf:
                for i, qr_img in enumerate(self.qrcodes):
                    img_path = os.path.join(temp_dir, f'page_{i+1}.png')
                    qr_img.save(img_path)
                    zipf.write(img_path, f'page_{i+1}.png')

            final_zip_path = os.path.join(save_path, "qrcodes.zip")
            os.rename(zip_path, final_zip_path)
            messagebox.showinfo("Download Complete", f"QR Codes downloaded successfully to {final_zip_path}")
        except Exception as e:
            messagebox.showerror("Error", str(e))

if __name__ == "__main__":
    root = tk.Tk()
    app = PDFtoQRCodeApp(root)
    root.mainloop()
